In [39]:
import datetime
import os

import constants

from azure.ai.ml import command, Input, Output
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential



In [40]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=constants.SUBSCRIPTION_ID,
    resource_group_name=constants.RESOURCE_GROUP_NAME,
    workspace_name=constants.WORKSPACE_NAME,
)

In [41]:
%%writefile components/prep.py 

from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url="https://mlgroupvault.vault.azure.net/", credential=credential)

import argparse

parser = argparse.ArgumentParser("prep")
parser.add_argument("--blob_storage", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--prep_output")
args = parser.parse_args()

# log in to the Blob Service Client
blob_storage = args.blob_storage
blob_storage_key = secret = secret_client.get_secret("blob_storage_key-name")
blob_service_client = BlobServiceClient(blob_storage, account_key=blob_storage_key.value)

# connect to the container 
container_client = blob_service_client.get_container_client(container="stock-news-json") 

# list and download all currently available blobs
blob_list = container_client.list_blobs()

# get the timestamp with the current day 
current_day_timestamp = datetime.datetime.today().timestamp()
current_day_timestamp = str(current_day_timestamp)[:8] # first 8 digits are the timestamp of the day

blobs_to_use = [blob.name for blob in blob_list if current_day_timestamp in blob.name]
for blob in blobs_to_use:
      print(f"Downloading blob: {blob}")
      blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob)
      with open(blob, mode="wb") as sample_blob:
            download_stream = blob_client.download_blob()
            sample_blob.write(download_stream.readall())

all_data_dict = {}
for json_file in blobs_to_use:
      with open(json_file,"r+") as file:
      # First we load existing data into a dict.
            file_data = json.load(file)
            all_data_dict.update(file_data)
with open("merged_stock_news.json", "w") as file:
      file.write(json.dumps(all_data_dict, indent=4))

# with open(args.blob_storage+"/blobs_to_use.txt", "w") as f:
#     f.write("\n".join(blob for blob in blobs_to_use), f)

(Path(args.prep_output) / "merged_stock_news.json")#.write_text("\n".join(blob for blob in blobs_to_use))
# continue here with this example -> https://github.com/Azure/azureml-examples/blob/main/sdk/python/jobs/pipelines/1a_pipeline_with_components_from_yaml/score_src/score.py

Overwriting components/prep.py


In [42]:
%%writefile components/classify.py

from pathlib import Path
import argparse
import json
import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification

parser = argparse.ArgumentParser()
parser.add_argument("--classify_input", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--classify_output", type=str, help="Mounted Azure ML blob storage")
args = parser.parse_args()

# download distilbert model from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("KernAI/stock-news-destilbert")
model = AutoModelForSequenceClassification.from_pretrained("KernAI/stock-news-destilbert")

# retriev the list of blobs from the current day - input is a .txt file
with open(args.classify_input, "r") as f:
      data = f.read()
texts = data["texts"]

sentiments = []
for text in texts: 
      tokenized_text = tokenizer(
            text,
            truncation=True,
            is_split_into_words=False,
            return_tensors="pt"
      )

      outputs = model(tokenized_text["input_ids"])
      outputs_logits = outputs.logits.argmax(1)

      mapping = {0: 'neutral', 1: 'negative', 2: 'positive'}
      predicted_label = mapping[int(outputs_logits[0])]
      sentiments.append(predicted_label)

# add the sentiments to the data
data["sentiments"] = sentiments

# overwrite old files with new files containing the sentiment
with open("merged_stock_news.json", "w") as f:
      json.dump(data, f)

(Path(args.prep_output) / "merged_stock_news.json")

Overwriting components/classify.py


In [43]:
%%writefile components/summarize.py

from pathlib import Path
import argparse
import json
import os

from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

parser = argparse.ArgumentParser()
parser.add_argument("--summarize_input", type=str, help="Mounted Azure ML blob storage")
parser.add_argument("--summarize_output", type=str, help="Mounted Azure ML blob storage")
args = parser.parse_args()

# load the model and the tokenizer
tokenizer = PegasusTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = PegasusForConditionalGeneration.from_pretrained("human-centered-summarization/financial-summarization-pegasus") 

# retriev the list of blobs from the current day - input is a .txt file
with open(args.summarize_input, "r") as f:
      data = f.read()
texts = data["texts"]

summaries = []
for text in texts: 
      # Tokenize our text
      # If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
      input_ids = tokenizer(text, return_tensors="pt").input_ids

      # Generate the output (Here, we use beam search but you can also use any other strategy you like)
      output = model.generate(
            input_ids, 
            max_length=32, 
            num_beams=5, 
            early_stopping=True
      )

      # Finally, we can print the generated summary
      summaries.append(tokenizer.decode(output[0], skip_special_tokens=True))

# add the sentiments to the data
data["summaries"] = summaries

# overwrite old files with new files containing the sentiment
with open("merged_stock_news.json", "w") as f:
      json.dump(data, f)

(Path(args.prep_output) / "merged_stock_news.json")

Overwriting components/summarize.py


In [44]:
%%writefile dependencies/conda.yml
name: stock-analysis-env
channels:
  - conda-forge
dependencies:
  - python=3.9
  - pip
  - pip:
    - azure-storage-blob
    - azure-identity
    - azure-keyvault
    - transformers
    - sentencepiece
    - numpy

Overwriting dependencies/conda.yml


In [45]:
custom_env_name = "stock-analysis-env"
version = "1.5"

try:    
    pipeline_job_env = ml_client.environments.get(custom_env_name, version=version)

except:
    pipeline_job_env = Environment(
        name=custom_env_name,
        description="Custom environment for stock analysis pipeline",
        conda_file=os.path.join("dependencies", "conda.yml"),
        image="mcr.microsoft.com/azureml/curated/python-sdk-v2:4",
        version=version,
    )
    pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

    print(
        f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
    )

Environment with name stock-analysis-env is registered to workspace, the environment version is 1.5


In [46]:
data_type = AssetTypes.URI_FOLDER
mode = InputOutputModes.RW_MOUNT
path = "azureml://datastores/stocknewsjson/stock-news-json"

In [47]:
data_prep_component = command(
    name="data_prep",
    display_name="Finding out which blobs to actually use",
    description="Loads files from Azure Blob Storage from todays ",
    inputs={
        "blob_storage": Input(mode=InputOutputModes.DIRECT)
    },
    outputs={
        "prep_output": Output(type=data_type)
    },
    code="./components/prep.py",
    command="python prep.py --blob_storage ${{inputs.blob_storage}} --prep_output ${{outputs.prep_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava"
)

In [48]:
classify_component = command(
    name="data_prep",
    display_name="Classify the sentiments of todays stock news",
    description="Loads data via AlphaVantage API input, preps data and stores to as data asset",
    inputs={
        "classify_input": Input(type=data_type), 
    },
    outputs={
        "classify_output": Output(type=data_type)
    },
    code="./components/classify.py",
    command="python classify.py --classify_input ${{inputs.classify_input}} --classify_output ${{outputs.classify_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava"
)

In [49]:
summarize_component = command(
    name="data_prep",
    display_name="Summarize the news",
    description="Loads data via AlphaVantage API input, preps data and stores to as data asset",
    inputs={
        "summarize_input": Input(type=data_type),
    },
    outputs={
        "summarize_output": Output(type=data_type, mode=InputOutputModes.RW_MOUNT, path=path)
    },
    code="./components/summarize.py",
    command="python summarize.py --summarize_input ${{inputs.summarize_input}} --summarize_output ${{outputs.summarize_output}}",
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    compute="ava"
)

In [50]:
from azure.ai.ml.dsl import pipeline

@pipeline(compute="ava")
def pipeline_with_python_function_components(account_url):

    data_prep_job = data_prep_component(
        blob_storage="https://mlstorageleo.blob.core.windows.net/stock-news-json"
    )
    classify_job = classify_component(
        classify_input=data_prep_job.outputs.prep_output

    ) # feed putput of previous step into the training job
    summarize_job = summarize_component(
        summarize_input = classify_job.outputs.classify_output
    )

    return {"processed_file": summarize_job.outputs.summarize_output}

pipeline_job = pipeline_with_python_function_components()

# set pipeline level compute
pipeline_job.settings.default_compute = "ava"

In [51]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="stock-news-analysis-pipeline"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
stock-news-analysis-pipeline,purple_lemon_k9sy690k1s,pipeline,Preparing,Link to Azure Machine Learning studio


In [52]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: purple_lemon_k9sy690k1s
Web View: https://ml.azure.com/runs/purple_lemon_k9sy690k1s?wsid=/subscriptions/5a361d37-b562-4eee-981b-0936493063e9/resourcegroups/MlGroup/workspaces/mlworkspace

Streaming logs/azureml/executionlogs.txt

[2023-06-29 18:19:52Z] Submitting 1 runs, first five are: 592623f0:bc27e84a-90a2-41ff-8f5e-9d510bb943fe
[2023-06-29 18:35:25Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: purple_lemon_k9sy690k1s
Web View: https://ml.azure.com/runs/purple_lemon_k9sy690k1s?wsid=/subscriptions/5a361d37-b562-4eee-981b-0936493063e9/resourcegroups/MlGroup/workspaces/mlworkspace


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. Failed nodes: /data_prep_job. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "northeurope",
    "location": "northeurope",
    "time": "2023-06-29T18:35:25.671782Z",
    "component_name": ""
} 